In [1]:
import pandas as pd
import numpy as np
import sklearn as sk

import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

import statsmodels.formula.api as smf

from sklearn.model_selection import cross_validate
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

C:\Users\anguy\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
game2016 = pd.read_csv('vgsales_ratings.csv') 

In [3]:
game2016.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#drop nan columns
game2016 = game2016.dropna()

#show sales in millions
game2016[['NA_Sales','EU_Sales','JP_Sales','Other_Sales','Global_Sales']]= game2016[['NA_Sales','EU_Sales','JP_Sales','Other_Sales','Global_Sales']]*1000000

game2016.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41360000.0,28960000.0,3770000.0,8450000.0,82530000.0,76.0,51.0,8,322.0,Nintendo,E
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15680000.0,12760000.0,3790000.0,3290000.0,35520000.0,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15610000.0,10930000.0,3280000.0,2950000.0,32770000.0,80.0,73.0,8,192.0,Nintendo,E
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11280000.0,9140000.0,6500000.0,2880000.0,29800000.0,89.0,65.0,8.5,431.0,Nintendo,E
7,Wii Play,Wii,2006.0,Misc,Nintendo,13960000.0,9180000.0,2930000.0,2840000.0,28920000.0,58.0,41.0,6.6,129.0,Nintendo,E


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

#effects of global sale and critic score

X = game2016[['Global_Sales']]
y = game2016['Critic_Score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
ridge_model = Ridge() #ridge regression

alphas = [0.001, 1, 10, 100] 

# Grid search with cross-validation to find the optimal alpha value for Ridge regression. 
#Negative mean squared error is used as the scoring metric for minimization.

grid_search = GridSearchCV(estimator=ridge_model, param_grid={'alpha':alphas}, scoring='neg_mean_squared_error', cv=5)

grid_search.fit(X_train, y_train)

#finding best alpha value through grid search
best_alpha = grid_search.best_params_['alpha']

#finding best RMSE. Converted to positive
grid_rmse = (-grid_search.best_score_)**0.5 #root mean squared error 

#ridge rmse after fitting with ridge regression model
ridge_model.fit(X_train, y_train)
predictions = ridge_model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = root_mean_squared_error(y_test, predictions)



print('Mean squared error from ridge_regression:', mse)
print('Root mean squared error from ridge regression: ', rmse)
print("Best alpha value:", best_alpha)
print("Best RMSE from cross-validation:", grid_rmse)

Mean squared error from ridge_regression: 185.25128195447803
Root mean squared error from ridge regression:  13.610704682509207
Best alpha value: 100
Best RMSE from cross-validation: 13.525567194900496


In [6]:
#using best alpha from above for ridge regression
ridge_model = Ridge(alpha = best_alpha)
ridge_model.fit(X_train, y_train)
y_test_pred = ridge_model.predict(X_test)
ridge_test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
print("Test RMSE for Ridge Regression:", ridge_test_rmse)

Test RMSE for Ridge Regression: 13.61070468250921


In [7]:
#global sales on user scores using ridge regression
X = game2016[['Global_Sales']]
y = game2016['User_Score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
ridge_model = Ridge() 

#finding best alpha value through grid search
alphas = [0.001, 1, 10, 100]

# Grid search with cross-validation to find the optimal alpha value for Ridge regression.
# Negative mean squared error is used as the scoring metric for minimization.

grid_search = GridSearchCV(estimator=ridge_model, param_grid={'alpha':alphas}, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

#best alpha value
best_alpha = grid_search.best_params_['alpha']

#finding best RMSE. Converted to positive
grid_rmse = (-grid_search.best_score_)**0.5

#ridge rmse after fitting with ridge regression model
ridge_model.fit(X_train, y_train)
predictions = ridge_model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = root_mean_squared_error(y_test, predictions)



print('Mean squared error from ridge_regression:', mse)
print('Root mean squared error from ridge regression: ', rmse)
print("Best alpha value:", best_alpha)
print("Best RMSE from cross-validation:", grid_rmse)

Mean squared error from ridge_regression: 2.146825437514343
Root mean squared error from ridge regression:  1.4652049131484453
Best alpha value: 0.001
Best RMSE from cross-validation: 1.4284123029943092


In [8]:
#using best alpha from above
ridge_model = Ridge(alpha = best_alpha)
ridge_model.fit(X_train, y_train)
y_test_pred = ridge_model.predict(X_test)
ridge_test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
print("Test RMSE for Ridge Regression:", ridge_test_rmse)

Test RMSE for Ridge Regression: 1.4652049131484453


In [9]:
#Linear regression using multiple factors to see effect on user score

import statsmodels.api as sm
import statsmodels.formula.api as smf

X = game2016[['Global_Sales', 'Genre', 'Platform', 'Publisher', 'Developer']]  
y = game2016['User_Score'].astype(float)

X_encoded = pd.get_dummies(X, drop_first=True)

X_num = X_encoded.astype(int) #turn numerical
model = sm.OLS(y, X_num)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:             User_Score   R-squared (uncentered):                   0.981
Model:                            OLS   Adj. R-squared (uncentered):              0.975
Method:                 Least Squares   F-statistic:                              174.4
Date:                Sun, 08 Dec 2024   Prob (F-statistic):                        0.00
Time:                        00:29:44   Log-Likelihood:                         -9837.5
No. Observations:                6825   AIC:                                  2.273e+04
Df Residuals:                    5296   BIC:                                  3.317e+04
Df Model:                        1529                                                  
Covariance Type:            nonrobust                                                  
==============================================================================================================================================================
                                                                                                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------------------------------------------
Global_Sales                                                                                4.871e-08   9.09e-09      5.360      0.000    3.09e-08    6.65e-08
Genre_Adventure                                                                                0.1223      0.116      1.057      0.290      -0.104       0.349
Genre_Fighting                                                                                 0.0549      0.100      0.547      0.584      -0.142       0.251
Genre_Misc                                                                                    -0.1280      0.094     -1.363      0.173      -0.312       0.056
Genre_Platform                                                                                 0.0826      0.084      0.988      0.323      -0.081       0.247
Genre_Puzzle                                                                                   0.1125      0.158      0.713      0.476      -0.197       0.422
Genre_Racing                                                                                   0.0941      0.091      1.031      0.303      -0.085       0.273
Genre_Role-Playing                                                                             0.3578      0.094      3.809      0.000       0.174       0.542
Genre_Shooter                                                                                  0.0079      0.075      0.106      0.916      -0.139       0.155
Genre_Simulation                                                                               0.3218      0.109      2.951      0.003       0.108       0.536
Genre_Sports                                                                                   0.0405      0.078      0.517      0.606      -0.113       0.194
Genre_Strategy                                                                                 0.4841      0.125      3.860      0.000       0.238       0.730
Platform_DC                                                                                    1.7187      0.362      4.742      0.000       1.008       2.429
Platform_DS                                                                                    0.6171      0.133      4.646      0.000       0.357       0.878
Platform_GBA                                                                                   1.2655      0.153      8.250      0.000       0.965       1.566
Platform_GC                                                                                    1.2188      0.138      8.83

In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

#multiple factors on user score done through ridge regression
X = game2016[['Global_Sales', 'Genre', 'Platform', 'Publisher', 'Developer']]  
y = game2016['User_Score']

X_encoded = pd.get_dummies(X, drop_first=True) #for non numerical data

#train test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

#ridge regression
ridge_model = Ridge()

alphas = [0.001, 1, 10, 100]

# Grid search with cross-validation to find the optimal alpha value for Ridge regression.
# Negative mean squared error is used as the scoring metric for minimization.

grid_search = GridSearchCV(estimator=ridge_model, param_grid={'alpha': alphas}, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

#finding best alpha value through grid search
best_alpha = grid_search.best_params_['alpha']

#finding best RMSE. Converted to positive
grid_rmse = (-grid_search.best_score_) ** 0.5


#best estimator through grid search and predictions to find mse and rmse
ridge_model = grid_search.best_estimator_
predictions = ridge_model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = root_mean_squared_error(y_test, predictions)

print('Mean squared error from ridge regression:', mse)
print('Root mean squared error from ridge regression: ', rmse)
print("Best alpha value:", best_alpha)
print("Best RMSE from cross-validation:", grid_rmse)


C:\Users\anguy\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.08367e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\anguy\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.11762e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\anguy\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=5.14591e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\anguy\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.28395e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\anguy\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:21

Mean squared error from ridge regression: 1.6679244232315635
Root mean squared error from ridge regression:  1.291481483890328
Best alpha value: 1
Best RMSE from cross-validation: 1.291633098746186


C:\Users\anguy\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.15039e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [11]:
#using best alpha from above
ridge_model = Ridge(alpha = best_alpha)
ridge_model.fit(X_train, y_train)
y_test_pred = ridge_model.predict(X_test)
ridge_test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
print("Test RMSE for Ridge Regression:", ridge_test_rmse)

Test RMSE for Ridge Regression: 1.291481483890328


C:\Users\anguy\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.15039e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [12]:
#multiple factors on critic score using ridge regression
X = game2016[['Global_Sales', 'Genre', 'Platform', 'Publisher', 'Developer']]  
y = game2016['Critic_Score']

X_encoded = pd.get_dummies(X, drop_first=True) #turn numerical 

#train test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

#ridge regression
ridge_model = Ridge()

alphas = [0.001, 1, 10, 100]

# Grid search with cross-validation to find the optimal alpha value for Ridge regression.
# Negative mean squared error is used as the scoring metric for minimization.

grid_search = GridSearchCV(estimator=ridge_model, param_grid={'alpha': alphas}, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

#finding best alpha through grid search
best_alpha = grid_search.best_params_['alpha']

#finding best RMSE. Converted to positive
grid_rmse = (-grid_search.best_score_) ** 0.5

#best estimator through grid search and predictions to find mse and rmse
ridge_model = grid_search.best_estimator_
predictions = ridge_model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
rmse = root_mean_squared_error(y_test, predictions)

print('Mean squared error from ridge regression:', mse)
print('Root mean squared error from ridge regression: ', rmse)
print("Best alpha value:", best_alpha)
print("Best RMSE from cross-validation:", grid_rmse)


C:\Users\anguy\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.08367e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\anguy\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.11762e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\anguy\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=5.14591e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\anguy\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.28395e-20): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\anguy\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:21

Mean squared error from ridge regression: 118.28719910172673
Root mean squared error from ridge regression:  10.875991867490832
Best alpha value: 1
Best RMSE from cross-validation: 11.12722831446973


C:\Users\anguy\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.15039e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [13]:
#using best alpha from above
ridge_model = Ridge(alpha = best_alpha)
ridge_model.fit(X_train, y_train)
y_test_pred = ridge_model.predict(X_test)
ridge_test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
print("Test RMSE for Ridge Regression:", ridge_test_rmse)

Test RMSE for Ridge Regression: 10.875991867490832


C:\Users\anguy\anaconda3\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.15039e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
